In [1]:
# installs

#!{sys.executable} -m pip install tqdm

# ниже - неудачная попытка отрисовать в three.js
#!curl -sL https://deb.nodesource.com/setup_12.x | sudo -E bash -
#!sudo apt update
#!sudo apt install -y nodejs
#!{sys.executable} -m pip install nodejs
#!{sys.executable} -m pip install pythreejs
# теперь можно слева во вкладе extensions установить jupyterlab-manager и threejs

In [2]:
# imports
import sys
import tqdm
from collections import Counter
import itertools

In [86]:
N = ToricLattice(7)
K = N([-3, 1, 1, 1, 1, 1, 1]) # canonical divisor
E = identity_matrix(QQ, 7)[:,1:].columns() # E is the set of exceptional curves, so the basis is L, E[0], .. , E[5]
Line = lambda exceptionals : (-K + sum(exceptionals)) / 3
L = Line(E)



Q = diagonal_matrix([1]+6*[-1])
dot = lambda a, b: a*Q*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])

minus_one_curves = E + [L-ei-ej for ei,ej in itertools.combinations(E, 2)] + [-K-L+e for e in E]
disjoint_curves = lambda curves: [c for c in minus_one_curves if all(dot(c,c2)==0 for c2 in curves)]

NE = Cone(minus_one_curves)
Ample = Cone([Q*ray for ray in NE.dual().rays()])


# take one Fujita face for each isomorphism class (determined by CPW, may check explicitly later)
# and turn them into subcones in Mori cone by adding -K

Fujita_face_rays_examples = {
        'B6': E,
        'B5': E[:5],
        'B5PP': E[:4]+[L-E[4]-E[5]], # B(5) for P1xP1. Another type of independent set of 5 (-1)-curves, this one is maximal
        'B4': E[:4],
        'B3': E[:3],
        'B2': E[:2],
        'B1': E[:1],
        #'C6': E[:5]+[L-E[5]-e for e in E[:5]], # C(6). Schlafli graph is 4-ultrahomogeneous, and, after fixing two pairs, three others are defined uniquely
        'C6B5': E[:5]+[L-E[5]], # здесь и ниже - части C6, из подобных которым складывается весь С6 
        'C6B4': E[:4]+[L-E[5]],
        'C6B3': E[:3]+[L-E[5]],
        'C6B2': E[:2]+[L-E[5]],
        'C6B1': E[:1]+[L-E[5]],
        'C6B0': E[:0]+[L-E[5]],
        'C6PP': E[:4]+[L-E[5]-E[4], L-E[5]],
    }
Fujita_cone_examples = {label:Cone(rays+[-K]).intersection(Ample) for label,rays in Fujita_face_rays_examples.items()}


# вычисляем типы граней
def face_type(face):
    m = gram_matrix(face.rays())
    eigenvalues = Counter(m.eigenvalues())
    #print(eigenvalues)
    if eigenvalues[-2]==5 and eigenvalues[0]==5:
        return "C"
    if len(eigenvalues.keys())!=1:
        print('ERROR face_type, eigenvalues: ',eigenvalues)
    if eigenvalues[-1]==5:
        if all(all(dot(curve,e)==0 for e in face.rays())==False for curve in minus_one_curves):
            return "B5PP" 
    return f"B{eigenvalues[-1]}"

assert face_type(Cone(Fujita_face_rays_examples['B5PP'])) == 'B5PP'

is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

def independent_sets(vectors, size = 6):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]

#cylinder1 = lambda E : E + [Line(E)-E[-1], -K-Line(E)+E[-1]]   
def cylinder2(E):
    '''
    E is a list of exceptional curves, and e is one of them
    returns 
    1. list of rays of the cone of divisors H such that the cylinder (P^2 minus conic through E-e and tangent line through e) is H-polar
    2. list of (-1)-curves lying in the complement of the cylinder (or a union of cylinders if many)
    '''
    for e in E:
        tangent = Line(E) - e
        conic = - K - tangent
        curves = E+[conic]
        yield curves + [tangent], curves

def cylinder1(E):
    """
    cylinder complement to E and lines through blowdowns of e and one of E-e
    """
    for e in E:
        curves = E+[Line(E)-e-f for f in E if f!=e]
        yield curves, curves

def cylinder_P1(E):
    '''
    CPW Example 4.1.6 and Lemma 4.2.2 for contraction of E1..E4, L-E5-E6.
    '''
    for i,j in itertools.permutations(range(len(E)),int(2)):
        complement = [k for k in range(len(E)) if (k!=i) and (k!=j)]
        conic = -K-Line(E)+E[i]
        curves = [E[k] for k in complement]+[Line(E)-E[i]-E[j], conic]
        yield curves + [Line(E)-E[i],Line(E)-E[j]], curves

def cylinder_cones(cylinder=cylinder2):
    '''
    returns a list of cones of compatible divisors for cylinders of considered type
    '''
    for blowdown in independent_sets(minus_one_curves):
        for ray_list,_ in cylinder(blowdown):
            yield Cone(ray_list)


In [7]:
# setting cones

cones = [cone for cone in cylinder_cones()] + [cone for cone in cylinder_cones(cylinder1)] + [cone for cone in cylinder_cones(cylinder_P1)]

#symmetries = [tuple(1 if i==j else -1 if i==j+1 else 0 for j in range(7)) for i in range(2,7)]
#ample_segment = Cone(list(ample_cone.dual().rays())+symmetries).dual()

#minkowski_sum = lambda cones : Cone([ray for cone in cones for ray in cone.rays()])
#contains_ample = lambda cone : all(cone.contains(ray) for ray in ample_cone.rays())
#is_subcone = lambda cone1, cone2 : all(cone2.contains(ray) for ray in cone1.rays())


In [87]:
shared_cylinders = lambda keys: [i for i,cone in enumerate(cones) if all(is_covered_by(Fujita_cone,cone) for key, Fujita_cone in Fujita_cone_examples.items() if key in keys) ]
for k in Fujita_cone_examples.keys():
    print(k,shared_cylinders([k]))



B6 [0, 1, 2, 3, 4, 5]
B5 [0, 1, 2, 3, 4, 5]
B5PP [864, 869, 900, 905, 930, 935, 1020, 1025, 1350, 1355]
B4 [2, 3, 4, 5]
B3 [3, 4, 5, 9, 10, 11]
B2 [4, 5, 10, 11, 16, 17, 22, 23, 28, 29, 71, 137]
B1 [5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95]
C6B5 [0]
C6B4 [869, 900, 930, 1020, 1350]
C6B3 [869, 874, 895, 900, 930, 960, 1020, 1050, 1350, 1380]
C6B2 [28, 29, 71, 137, 869, 874, 879, 895, 900, 911, 925, 930, 941, 955, 960, 971, 1020, 1050, 1110, 1334, 1350, 1380, 1440, 1664]
C6B1 [29, 47, 59, 65, 71, 77, 83, 89, 869, 874, 879, 884, 895, 900, 911, 916, 925, 930, 941, 946, 955, 960, 971, 976, 1015, 1020, 1031, 1036, 1045, 1050, 1061, 1066, 1105, 1110, 1121, 1126, 1319, 1324, 1329, 1334, 1350, 1380, 1440, 1664, 1680, 1904, 2084, 2234]
C6B0 [869, 874, 879, 884, 889, 895, 900, 911, 916, 921, 925, 930, 941, 946, 951, 955, 960, 971, 976, 981, 1015, 1020, 1031, 1036, 1041, 1045, 1050, 1061, 1066, 1071, 1105, 1110, 1121, 1126, 1131, 1319, 1324, 1329, 1334, 1339, 1345, 1350, 1361,

In [36]:
    
k = ['B6','B5','B4']
print(k,shared_cylinders(k))

k = ['B3','B2','B1']
print(k,shared_cylinders(k))

k = ['B5PP','C6B4','C6B3','C6B2','C6B1','C6B0','C6PP']
print(k,shared_cylinders(k))


['B6', 'B5', 'B4'] [2, 3, 4, 5]
['B3', 'B2', 'B1'] [5, 11]
['B5PP', 'C6B4', 'C6B3', 'C6B2', 'C6B1', 'C6B0', 'C6PP'] [869, 900, 930, 1020, 1350]


In [88]:
curves_in_complement = lambda cylinder: [i for i, curve in enumerate(minus_one_curves) if N(curve) in cylinder.rays()]
def shared_curves(cylinders): 
    if len(cylinders)==0:
        return minus_one_curves
    complements = curves_in_complement(cylinders[0])
    shared_complements = [curve for curve in complements if all(curve in curves_in_complement(cylinder) for cylinder in cylinders[1:])]
    return shared_complements

complement_cone = lambda cylinders: Cone([minus_one_curves[i]  for i in shared_curves(cylinders)]).intersection(Ample)

for k in Fujita_cone_examples.keys():
    print(k,complement_cone([cones[i] for i in shared_cylinders([k])]))
#def shared_lines(cylinders)
#cones[0].rays()
#curves_in_complement(cones[0])
#minus_one_curves

B6 0-d cone in 7-d lattice N
B5 0-d cone in 7-d lattice N
B5PP 0-d cone in 7-d lattice N
B4 0-d cone in 7-d lattice N
B3 0-d cone in 7-d lattice N
B2 0-d cone in 7-d lattice N
B1 0-d cone in 7-d lattice N
C6B5 6-d cone in 7-d lattice N
C6B4 6-d cone in 7-d lattice N
C6B3 0-d cone in 7-d lattice N
C6B2 0-d cone in 7-d lattice N
C6B1 0-d cone in 7-d lattice N


ValueError: lattice must be given explicitly if there are no rays!

In [13]:
# studying bad ray
N = ToricLattice(7)
badray = N(51, -26, -13, -7, -4, -2, -1)
print(ample_cone.contains(badray))
for face in effective_cone.faces()[6]:
    intersection = face.intersection(Cone([badray,K]))
    if intersection.dim() == 0:
        continue
    else:
        break
print(intersection.rays())
print(face.rays())
ray = intersection.rays()[0]
print(face.relative_interior_contains(ray))
e1, e2, e3, e4, e5, f1 = face.rays()[:6]
b = e1 + f1
R = 63*e1 + 59*e2 + 54*e3 + 50*e4 + 19*e5
print(R)
any(cone.interior_contains(badray) for cone in cones)
# its Fujita face is spanned by rays L-E1-Ej and L-\sum(Ei)+Ej, where j=2..6, the class K+\mu*H equals ray/12.
# they comprise 5 pairs 
# its Fujita map is of type C(6)

True
N(27, -27, -1, 11, 17, 21, 23)
in 7-d lattice N
N(0,  0,  1,  0,  0,  0,  0),
N(0,  0,  0,  1,  0,  0,  0),
N(0,  0,  0,  0,  1,  0,  0),
N(0,  0,  0,  0,  0,  1,  0),
N(0,  0,  0,  0,  0,  0,  1),
N(1, -1, -1,  0,  0,  0,  0),
N(1, -1,  0, -1,  0,  0,  0),
N(1, -1,  0,  0, -1,  0,  0),
N(1, -1,  0,  0,  0, -1,  0),
N(1, -1,  0,  0,  0,  0, -1)
in 7-d lattice N
True
N(0, 0, 63, 59, 54, 50, 19)


False

In [ ]:
# dissections 

dissecting_planes = lambda cones : [ray for cone in cones for ray in cone.dual().rays() if not ample_segment.dual().contains(ray)]

def dissect_with_plane(cones, plane):
    dissected = []
    for cone in cones:
        #print(f'dissecting with plane, {len(dissected)} cones')
        if cone.dual().contains(plane) or cone.dual().contains(-plane):
            dissected.append(cone)
        else:
            dual_rays = list(cone.dual().rays())
            positive_part = Cone(dual_rays + [plane]).dual()
            negative_part = Cone(dual_rays + [-plane]).dual()
            dissected.append(positive_part)            
            dissected.append(negative_part)            
    return dissected

def dissect_with_planes(cones, planes):
    dissected = cones
    for i,p in enumerate(planes):
        print(f'dissecting with {i}-th plane of {len(planes)}, {len(dissected)} cones')
        dissected = dissect_with_plane(dissected, p)
    return dissected

dissects = lambda cone, plane: not( cone.dual().contains(plane) or cone.dual().contains(-plane) )


def halves(cone, plane):
    dual_rays = list(cone.dual().rays())
    negative_part = Cone(dual_rays + [-plane]).dual()
    positive_part = Cone(dual_rays + [plane]).dual()
    return negative_part, positive_part

def dissect_wrt(dissected_cone, dissecting_cone): # intersection is returned separately
    parts = []
    remainder = dissected_cone
    for plane in dissecting_cone.dual().rays():
        if not dissects(remainder, plane):
            continue
        negative_part, positive_part = halves(remainder, plane)
        parts.append(negative_part)
        remainder = positive_part
    if remainder.intersection(dissecting_cone).is_solid():
        return parts, remainder
    else:
        parts.append(remainder)    
        return parts, None

#[c.rays() for c in dissect_wrt(Cone([[1,0],[0,1]]),Cone([[1,1],[-1,1]]))]

class Pseudofan: # solid version
    def __init__(self, cone):
        self.cones = [cone]
        #for cone in cones:
        #    self.append(cone)
    
    def remove(self, cone):
        #print('removing cone\n',cone.rays())
        outer_parts = [dissect_wrt(old, cone)[0] for old in self.cones]
        #print('outer parts\n', outer_parts)
        self.cones  = list(itertools.chain(*outer_parts))
        #print('cones\n', self.cones)

    def intersect(self, cone):
        intersections = [cone.intersection(old) for old in self.cones]
        self.cones = [c for c in intersections if c.is_solid()]


def remove_cones(pseudofan, cones_list, cap = None):
    pbar = tqdm(cones_list)
    for cone in pbar:
        pseudofan.remove(cone)
        pbar.set_description(f'{len(pseudofan.cones)} cones in pseudofan ')
        if cap:
            P.cones = P.cones[:cap]

In [ ]:
C1, C2 = Cone([[1,0],[0,1]]), Cone([[1,1],[-1,1]])
#F1, F2 = Fan([C1]), Fan([C2])
#F1.common_refinement(F2)
P = Pseudofan(C2)
P.remove(C1)
P.remove(C2)
for c in P.cones:
    print(c.rays())    

In [ ]:
P = Pseudofan(ample_segment)        
remove_cones(P, filtered_cones)
remove_cones(P, cones[::-1], cap=100)
Counter([c.dim() for c in P.cones])
bad_rays = [sum(c.rays()) for c in P.cones]
pbar = tqdm(bad_rays)
#pbar.set_description("Processing %s" % char)
for ray in pbar:
    if not any(c.contains(ray) for c in cones):
        print(ray)

In [ ]:
N = ToricLattice(3)
e1, e2, e3 = N([1,0,0]), N([0,1,0]), N([0,0,1])
r = e1+e2+e3
P3 = Pseudofan(Cone([e1,e2,e3]))
P3.remove(Cone([r,e2,e3]))
P3.remove(Cone([e1,r,e3]))
P3.remove(Cone([e1,r,r]))
for c in P3.cones:
    print(c.rays())
    #c.plot().show()

In [41]:
curves = Fujita_face_rays_examples['B5P1'] #[E[1], E[2], L-E[0]-E[1], L-E[0]-E[2]]
[c for c in minus_one_curves if all(dot(c,e)==1 for e in curves)]

[(2, -1, -1, -1, -1, 0, -1), (2, -1, -1, -1, -1, -1, 0)]

In [ ]:
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))
#[cone.rays() for cone in cones if Fujita_cone_examples['B5P1'].intersection(cone).dim()]
remove_cones(Pseudofan(Fujita_cone_examples['B5P1']), cones)


In [ ]:
A=Fujita_cone_examples['B5P1']
A.intersection(A)
A.dim()
